In [ ]:
import pandas as pd
import re
from difflib import SequenceMatcher
import os  # Para manipular rutas y nombres de archivos

# Función para normalizar nombres y ordenar las palabras
def normalize_and_sort_name(name):
    if isinstance(name, str):
        # Convertir a minúsculas
        name = name.lower()
        # Reemplazar variaciones comunes
        name = re.sub(r'[vb]', 'b', name)
        name = re.sub(r'[scz]', 's', name)
        name = re.sub(r'[iy]', 'i', name)
        # Eliminar espacios adicionales
        name = re.sub(r'\s+', ' ', name).strip()
        # Dividir en palabras y ordenar
        name_parts = name.split()
        name_parts.sort()  # Ordenar alfabéticamente
        return ' '.join(name_parts)  # Unir las palabras ordenadas
    return name  # Devolver el valor original si no es una cadena

# Función para calcular la similitud entre dos cadenas
def similarity_ratio(name1, name2):
    return SequenceMatcher(None, name1, name2).ratio()

# Establecer el umbral de similitud
similarity_threshold = 0.80

# Función para encontrar coincidencias aproximadas en la base
def find_match_in_base(solicitud_name, base_names):
    if not isinstance(solicitud_name, str):
        return "No encontrado"
    for base_name in base_names:
        if similarity_ratio(solicitud_name, base_name) >= similarity_threshold:
            return "Encontrado"
    return "No encontrado"

# Cargar los archivos Excel
base_file_path = 'Archivo1.xlsx'
solicitud_file_path = 'Archivo2.xlsx'

# Extraer el nombre del archivo sin la extensión
solicitud_filename = os.path.splitext(os.path.basename(solicitud_file_path))[0]

base_df = pd.read_excel(base_file_path)
solicitud_df = pd.read_excel(solicitud_file_path)

# Normalizar y ordenar los nombres en ambas bases
base_df['normalized_name'] = base_df['APELLIDOS Y NOMBRES'].apply(normalize_and_sort_name)
solicitud_df['normalized_name'] = solicitud_df['NOMBRES Y APELLIDOS'].apply(normalize_and_sort_name)

# Aplica la comparación
base_names = base_df['normalized_name'].tolist()
solicitud_df['Resultado Comparación'] = solicitud_df['normalized_name'].apply(lambda x: find_match_in_base(x, base_names))

# Construir el nombre del archivo de salida anteponiendo "Resultados"
output_file_path = f"Resultados_{solicitud_filename}.xlsx"

# Guarda el archivo con el resultado de la comparación
solicitud_df.to_excel(output_file_path, index=False)

print(f"Archivo guardado en: {output_file_path}")
